[statinfer.com](https://statinfer.com/)

# Packages

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import scipy as sp
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import time
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn import preprocessing

# Data Importing

Dataset Background : https://www.kaggle.com/c/otto-group-product-classification-challenge/notebooks

In [2]:
ecom_prod_train=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Ecom_Products_Menu/train.csv")
ecom_prod_test=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Ecom_Products_Menu/test.csv")

In [3]:
print(ecom_prod_train.sample(10))
print(ecom_prod_test.sample(10))

          id  spec1  spec2  spec3  ...  spec98  spec99  spec100       Category
6224    7689      0      0      1  ...       2       0        0        Tablets
6252    7726      0      0      0  ...       0       0        0        Tablets
22417  27640      0      0      0  ...       0       1        0             TV
45886  56589      0      1      0  ...       0       0        0     Appliances
3677    4538      0      0      0  ...       0       0        0        Tablets
44345  54660      0      0      0  ...       0       0        0     Appliances
45903  56607      0      0      0  ...       1       0        0     Appliances
17481  21554      0      0      0  ...       0       0        0        Laptops
16028  19775      0      0      0  ...       0       0        0        Laptops
25978  32077      0      0      0  ...       0       0        0  Personal_Care

[10 rows x 102 columns]
          id  spec1  spec2  spec3  ...  spec98  spec99  spec100    Category
5579   29310      0      0    

In [4]:
#Frequency counts of the products
ecom_prod_train['Category'].value_counts()

Tablets          13079
Personal_Care    11452
Appliances        6863
Laptops           6466
Camara            3964
Accessories       2341
Ipod              2217
TV                2182
Mobiles           1558
Name: Category, dtype: int64

# Prepare Features, Train and Test data

In [5]:
features=list(ecom_prod_train.columns[1:101])
print("features \n", features)
X_train=ecom_prod_train[features]
y_train=ecom_prod_train['Category']

X_test=ecom_prod_test[features]
y_test=ecom_prod_test['Category']

features 
 ['spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6', 'spec7', 'spec8', 'spec9', 'spec10', 'spec11', 'spec12', 'spec13', 'spec14', 'spec15', 'spec16', 'spec17', 'spec18', 'spec19', 'spec20', 'spec21', 'spec22', 'spec23', 'spec24', 'spec25', 'spec26', 'spec27', 'spec28', 'spec29', 'spec30', 'spec31', 'spec32', 'spec33', 'spec34', 'spec35', 'spec36', 'spec37', 'spec38', 'spec39', 'spec40', 'spec41', 'spec42', 'spec43', 'spec44', 'spec45', 'spec46', 'spec47', 'spec48', 'spec49', 'spec50', 'spec51', 'spec52', 'spec53', 'spec54', 'spec55', 'spec56', 'spec57', 'spec58', 'spec59', 'spec60', 'spec61', 'spec62', 'spec63', 'spec64', 'spec65', 'spec66', 'spec67', 'spec68', 'spec69', 'spec70', 'spec71', 'spec72', 'spec73', 'spec74', 'spec75', 'spec76', 'spec77', 'spec78', 'spec79', 'spec80', 'spec81', 'spec82', 'spec83', 'spec84', 'spec85', 'spec86', 'spec87', 'spec88', 'spec89', 'spec90', 'spec91', 'spec92', 'spec93', 'spec94', 'spec95', 'spec96', 'spec97', 'spec98', 'spec99', 'spec10

# Decision Tree model

In [ ]:
###buildng Decision tree on the training data ####
from sklearn import tree
tree = tree.DecisionTreeClassifier(max_depth=9)
tree.fit(X_train,y_train)

## Decision Tree Results

In [ ]:
#Accuracy on train data
print("Decision Tree Results \n")
print("Accuracy on train data" , tree.score(X_train, y_train))
print("Accuracy on test data" , tree.score(X_test, y_test))

# Boosted Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
boost=GradientBoostingClassifier(n_estimators=100,verbose=1) 
##fitting the gradient boost classifier
start_time = time.time()
boost.fit(X_train,y_train)
print("Time taken by GBM "+ str((time.time() - start_time))+ " Seconds")

## GBM Results

In [ ]:
###predicting Gradient boosting model on the train Data
boost_predict_train=boost.predict(X_train)
cm1 = confusion_matrix(y_train,boost_predict_train)
print(cm1)

accuracy_train=f1_score(y_train, boost_predict_train, average='micro') 
print("train accuracy", accuracy_train)

In [ ]:
###predicting Gradient boosting model on the test Data
boost_predict_test=boost.predict(X_test)
cm1 = confusion_matrix(y_test,boost_predict_test)
print(cm1)

accuracy_test=f1_score(y_test, boost_predict_test, average='micro') 
print("test accuracy", accuracy_test)

# XGB Model

In [6]:
#Creating XGB Friendly data and matrices
train_labels = y_train.values
train_labels = preprocessing.LabelEncoder().fit_transform(train_labels)
test_labels = y_test.values
test_labels = preprocessing.LabelEncoder().fit_transform(test_labels)

matrix_train = xgboost.DMatrix(X_train,label=train_labels)
matrix_test = xgboost.DMatrix(X_test,label=test_labels)

In [7]:
params = {
    'max_depth': 8, 
    'eta':0.02, #Learning Rate
    'eval_metric':'merror', # Multiclass classification error rate. 
    'tree_method' : "gpu_hist", # use this with colab in gpu mode for faster training
    'num_class': 9
}

start_time = time.time()

model=xgboost.train(params=params,
                    dtrain=matrix_train,
                    num_boost_round=300,    #Number of trees
                    early_stopping_rounds=3, # Stop after 3 rounds, if test error doesn't improve. 
                    evals=[(matrix_test,'test')] 
                   )

print("Time taken by XGB "+ str((time.time() - start_time))+ " Seconds")

[0]	test-merror:0.262079
Will train until test-merror hasn't improved in 3 rounds.
[1]	test-merror:0.258676
[2]	test-merror:0.253828
[3]	test-merror:0.254338
[4]	test-merror:0.249319
[5]	test-merror:0.247108
[6]	test-merror:0.246853
[7]	test-merror:0.246342
[8]	test-merror:0.245407
[9]	test-merror:0.245322
[10]	test-merror:0.24345
[11]	test-merror:0.241834
[12]	test-merror:0.240473
[13]	test-merror:0.240728
[14]	test-merror:0.240473
[15]	test-merror:0.240303
[16]	test-merror:0.239197
[17]	test-merror:0.239112
[18]	test-merror:0.238687
[19]	test-merror:0.237751
[20]	test-merror:0.237155
[21]	test-merror:0.237326
[22]	test-merror:0.235199
[23]	test-merror:0.234263
[24]	test-merror:0.233498
[25]	test-merror:0.233158
[26]	test-merror:0.232987
[27]	test-merror:0.231286
[28]	test-merror:0.231031
[29]	test-merror:0.231116
[30]	test-merror:0.231201
[31]	test-merror:0.230946
[32]	test-merror:0.230606
[33]	test-merror:0.229925
[34]	test-merror:0.22933
[35]	test-merror:0.22916
[36]	test-merror:0.

## XGB Results

In [ ]:
###prediction using XGB on the train Data
boost_predict_train=model.predict(matrix_train)
cm1 = confusion_matrix(train_labels,boost_predict_train)
print(cm1)

accuracy_train=f1_score(train_labels, boost_predict_train, average='micro') 
print("train accuracy", accuracy_train)

In [ ]:
###prediction using XGB on the test Data
boost_predict_test=model.predict(matrix_test)
cm1 = confusion_matrix(test_labels,boost_predict_test)
print(cm1)

accuracy_test=f1_score(test_labels, boost_predict_test, average='micro') 
print("test accuracy", accuracy_test)